# 구글플레이에 등록된 앱의 리뷰 추출

In [1]:
# 기본적으로 불러와야되는 셀레니윰 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

#사람인척 하기 위한 옵션으로 사람이 아닌 셀레니움이면 사이트에서 막히기 때문에 사용
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/141.0.0.0")
options.add_argument("lang=ko_KR")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )
#웹브라우저에서 url열기
driver.get("https://play.google.com/store/apps/details?id=viva.republica.toss&hl=ko")

In [2]:
import time

In [3]:
driver.execute_script(f"window.scrollTo({0},{1400})")
time.sleep(2)

In [4]:
# 평점 및 리뷰 옆의 -> 버튼 클릭
button = driver.find_element(By.CSS_SELECTOR,'button[aria-label*="평점 및 리뷰 자세히 알아보기"]')
button.click()

In [5]:
# 최신순으로 리뷰를 정리하기 위해서 버튼 클릭
driver.find_element(By.ID,"sortBy_1").click()

In [6]:
# 최신을 찾아서 클릭
driver.find_element(By.CSS_SELECTOR,'span[aria-label*="최신"]').click()

In [7]:
# window를 스크롤하면 리뷰가 아닌 배경 페이지가 스크롤 됨

In [8]:
# 리뷰가 담긴 창을 찾아서 Javascript로 1000px씩 아래로 스크롤
driver.execute_script("document.querySelector('.fysCi.Vk3ZVd').scrollBy(0,10000)")

In [9]:
#리뷰일
review_date = driver.find_element(By.CSS_SELECTOR,".bp9Aid").get_attribute('innerHTML')

In [10]:
# 별점
rating = float(driver.find_element(By.CSS_SELECTOR,'div[aria-label*="별표 5개 만점에"]').get_attribute('aria-label').split()[3].replace("개를",""))

In [54]:
#사용자 리뷰
user_reviwe = driver.find_element(By.CSS_SELECTOR, ".h3YV2d").text

In [445]:
# 회사 응답
company_reply = driver.find_element(By.CSS_SELECTOR, ".ras4vb").text

In [81]:
from datetime import datetime,timedelta

In [72]:
# 추출한 리뷰일을 날짜형 데이터로 변경
# datetime.strptime(yyyy-mm-dd, '%Y-%m-%d') 날짜형 데이터 타입으로 변환
review_date = review_date.replace(" ","").replace("년","-").replace("월","-").replace("일","")
review_date = datetime.strptime(review_date,'%Y-%m-%d')

In [73]:
review_date

datetime.datetime(2025, 10, 1, 0, 0)

In [82]:
today = datetime.today()
one_month_ago = today - timedelta(days=30)
one_month_ago.date()

datetime.date(2025, 9, 27)

# 토스 정보 합치기

In [36]:
pip install PyMySQL

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 웹 정보 불러올때까지 기달려주는 코드
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

웹브라우저에 표시되는 요소들(버튼,아이콘)은 클릭할 수 있게 되기까지 시간이 필요. <br>
요소가 클릭이 가능해질 때 까지 기다리도록 코드를 작성해 주어야 함<br>
```python
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
```
추가가 필요

In [1]:
import os
import time
import pymysql
import pandas as pd
from dbio import load_data, to_db,db_connect
from io import StringIO
from selenium import webdriver
from datetime import datetime,timedelta
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

root


In [2]:
def to_date(review_date):
    review_date = review_date.replace(" ","").replace("년","-").replace("월","-").replace("일","")
    review_date = datetime.strptime(review_date,'%Y-%m-%d')
    return review_date.date()

In [3]:
apps = dict (토스 = "viva.republica.toss&hl=ko", 국민 = "com.kbstar.kbbank&hl=ko",하나 = "com.kebhana.hanapush&hl=ko",
            뱅크셀러드 = "com.rainist.banksalad2&hl=ko0",핀다 = "kr.co.finda.finda&hl=ko")

In [4]:
def app_teview_extractor(apps):
    #사람인척 하기 위한 옵션으로 사람이 아닌 셀레니움이면 사이트에서 막히기 때문에 사용
    options = Options()
    options.add_experimental_option("detach", True)
    options.add_argument("start-maximized")
    options.add_argument("Chrome/141.0.0.0")
    options.add_argument("lang=ko_KR")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )
    #웹브라우저에서 url열기
    driver.get(f"https://play.google.com/store/apps/details?id={app[1]}")

    # 요소가 실행 가능해질 때 까지 기다리기(기다리는 최대 시간)
    wait = WebDriverWait(driver,10)
    # 평점 및 리뷰 옆의 -> 버튼이 활성화 되도록 스크롤을 1400px 아래로 내림
    driver.execute_script(f"window.scrollTo({0},{1400})")
    time.sleep(2)

    # 평점 및 리뷰 옆의 -> 버튼 클릭
    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'button[aria-label*="평점 및 리뷰 자세히 알아보기"]')))
    button.click()

    # 최신순으로 리뷰를 정리하기 위해서 버튼 클릭
    wait.until(EC.element_to_be_clickable((By.ID, "sortBy_1"))).click()
    time.sleep(2)

    # 최신을 찾아서 클릭
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[aria-label*="최신"]'))).click()
    time.sleep(2)
    #스크롤을 내린 후 마지막 자료의 날짜를 찾아서 오늘 날짜부터 1주일 전 날짜와 비교하기
    # 첫페이지 마지막 리뷰의 날짜 구하기 오늘 날짜랑 비교하기
    #스크롤 전과 후의 리뷰 개수 비교하기
    today = datetime.today()
    today = today.date()
    end_date = today - timedelta(days=7)
    end_date_str = end_date.strftime('%Y년 %m월 %d일')
    to_date(end_date_str)
    end_px = 1000
    len_review_list = len(driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe"))
    last_review = driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe")[-1]
    last_review_date = to_date(last_review.find_element(By.CSS_SELECTOR,".bp9Aid").get_attribute('innerHTML'))

    while last_review_date >= end_date:
        driver.execute_script(f"document.querySelector('.fysCi.Vk3ZVd').scrollTo(0,{end_px})")
        time.sleep(2)
        last_review = driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe")[-1]
        last_review_date = to_date(last_review.find_element(By.CSS_SELECTOR,".bp9Aid").get_attribute('innerHTML'))

        end_px += 10000
        print(len( driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe")))
        print(f"last_review_date:{last_review_date},end_date:{end_date}")
    # 리뷰 목록 가져오기 
    review_list = driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe")

    result = {}
    cols = ('리뷰일','앱 이름','별점','사용자 리뷰','회사 응답')
    r = 10000

    for review in review_list:
        #리뷰일
        review_date = review.find_element(By.CSS_SELECTOR,".bp9Aid").get_attribute('innerHTML')
        review_date = review_date.replace(" ","").replace("년","-").replace("월","-").replace("일","")
        # 별점
        rating = float(review.find_element(By.CSS_SELECTOR,'div[aria-label*="별표 5개 만점에"]').get_attribute('aria-label').split()[3].replace("개를",""))

        #사용자 리뷰
        user_reviwe = review.find_element(By.CSS_SELECTOR, ".h3YV2d").text

        # 회사 응답
        try:
            company_reply = review.find_element(By.CSS_SELECTOR, ".ras4vb > div").get_attribute("innerHTML")
        except:
            company_reply = "회사응답 없음"
            
        values = (review_date,apps[0],rating,user_reviwe,company_reply)
        for key,value in zip(cols,values):
            result.setdefault(key,[]).append(value)
            
    driver.close()        
    df = pd.DataFrame(result)
    return df

In [5]:
for app in apps.items():
    df = app_teview_extractor(app)
    display(df)
    df.to_csv(f"./data/{app[0]}_앱 리뷰 모음.csv",index = False , encoding = "utf-8-sig")
    to_db("fintech_app","app_review",df)

20
last_review_date:2025-10-27,end_date:2025-10-22
40
last_review_date:2025-10-26,end_date:2025-10-22
60
last_review_date:2025-10-25,end_date:2025-10-22
80
last_review_date:2025-10-24,end_date:2025-10-22
100
last_review_date:2025-10-23,end_date:2025-10-22
120
last_review_date:2025-10-22,end_date:2025-10-22
140
last_review_date:2025-10-21,end_date:2025-10-22


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025-10-27,토스,5.0,수수료가 0원,회사응답 없음
1,2025-10-27,토스,1.0,미쳤냐 소리가 절로나오네 어제 주식창 계속 멈춤 돈도많이번다매 서버증축좀 해라 난갈아탄다,"안녕하세요. 꿀언니님, 토스팀입니다. 우선 토스증권 이용에 불편을 드려 죄송하다는 ..."
2,2025-10-27,토스,5.0,전화기 업데이트하고 나니 이런 문구가 계속 떠요 토스에 필요한 조치을 완료해주세요 ...,"안녕하세요. 장미란 님, 토스팀입니다. 먼저 서비스 이용에 불편을 드렸다면 대단히 ..."
3,2025-10-27,토스,5.0,"다 좋은데, 나의 자산 부분에 코인 관련 정보도 추가할 수 있었으면 좋겠어요","안녕하세요. taeng 님, 토스팀입니다. 토스를 이용하시면서 느끼신 소중한 의견을..."
4,2025-10-27,토스,1.0,토스가편해서사용하는데 다른어플로갈아타고싶네요,"안녕하세요. 진혜련 님, 토스팀입니다. 먼저 서비스 이용에 불편을 드렸다면 대단히 ..."
...,...,...,...,...,...
135,2025-10-21,토스,1.0,현재 앱이 최악이에요 혜택에 이런저런 서비스를 넣은 건 좋은데 어제부터 계속 앱이 ...,"안녕하세요. 권혜미님, 토스팀입니다.우선 토스 앱 및 서비스 이용에 불편을 드려 정..."
136,2025-10-21,토스,3.0,만보기 전부 권한 허용해도 걸음수 측정을 못합니다,"안녕하세요. 치즈불닭라면님, 토스팀입니다. 이용에 불편을 드려 죄송합니다. 걸음수 ..."
137,2025-10-21,토스,3.0,좋긴한데 카드 밑에 깨진것 같은 구멍은 왜 있고 일반 블랙은 어디갔나요? 구멍 때문...,회사응답 없음
138,2025-10-21,토스,2.0,홍민택 데려가주세요 카카오톡 돌려내주세요,회사응답 없음


20ntech_app 데이터베이스 확인/생성완료
last_review_date:2025-10-24,end_date:2025-10-22
40
last_review_date:2025-10-22,end_date:2025-10-22
60
last_review_date:2025-10-20,end_date:2025-10-22


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025-10-27,국민,1.0,행안부 모바일 앱 안쓰고 국민은행 앱에서 쓰려고 하는거지 행안부 앱이 필수라고 생각...,"안녕하세요, 고객님. KB스타뱅킹 이용에 불편을 드려 죄송합니다. KB스타뱅킹 국민..."
1,2025-10-27,국민,1.0,얼굴인식 안되는데 계속 촬영하라고 해요. 너무 불편합니다.,회사응답 없음
2,2025-10-27,국민,1.0,이런...은행어플은 또... 새롭게 이용자를 빡치게하네요. 신분증 인식하는데 50번...,회사응답 없음
3,2025-10-27,국민,5.0,간편하고 편합니다.,회사응답 없음
4,2025-10-27,국민,5.0,굿굿굿 편해요 눈 침침한 저에게 큰글씨 있어서 편함,회사응답 없음
5,2025-10-27,국민,4.0,굿,회사응답 없음
6,2025-10-27,국민,5.0,사용하기가 편해요,회사응답 없음
7,2025-10-26,국민,5.0,메뉴들이 상세한 부분이 있음,고객님 안녕하세요! KB스타뱅킹을 이용해 주시고 바쁘신 와중에 칭찬글까지 남겨주시어...
8,2025-10-26,국민,5.0,고객센터 알려준데로 하니 쉽게했네요~,"안녕하세요, 고객님. KB스타뱅킹을 꾸준히 이용해주시는 고객님께 진심으로 감사의 말..."
9,2025-10-26,국민,5.0,최고,"안녕하세요, 고객님. KB스타뱅킹을 이용해주시고, 바쁘신 시간 중에도 소중한 칭찬글..."


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025-10-27,하나,1.0,진짜 이렇게 조잡한 은행 어플은 처음이네 이체 실패한 메시지가 팝업되면 입력하던 화...,회사응답 없음
1,2025-10-27,하나,1.0,별하나도 아깝네요 간편인증에서 넘어가지 않고 클릭도 안되고 먹통이 됩니다 대대적인 ...,회사응답 없음
2,2025-10-27,하나,1.0,임의 개조 OS 어쩌고로 접속 막는거 최신버전에서도 개선 전혀 안됐는데 열심히 복사...,회사응답 없음
3,2025-10-27,하나,5.0,사용할때마다 편리하고 안전하다고 생각됩니다,회사응답 없음
4,2025-10-27,하나,4.0,불편,회사응답 없음
5,2025-10-27,하나,1.0,하나은행은 왜 계좌이체도 안 돼요!,회사응답 없음
6,2025-10-27,하나,1.0,버그버그 내계좌이체시 무한로딩입니다. 오류코드 BEBK99999,회사응답 없음
7,2025-10-26,하나,1.0,잘 쓰고 있었는데 이번달부터 갑자기 개조된 OS에서 사용이 불가하다고 강제 종료 됩...,회사응답 없음
8,2025-10-24,하나,1.0,예수금 빼려고 보니까 그냥은 또 안빠지고 계좌 밀어야 된데서 계좌 해지 찾아보니 밀...,회사응답 없음
9,2025-10-24,하나,1.0,앱이 왜 안열리져 다른통장에서 이체를해서 볼려고해도볼수가없네여 출근 교통비 써야되는...,회사응답 없음


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025-10-27,뱅크셀러드,5.0,코인도 자산 연동해서 갱신할수있게 해주세요. 수동입력 말구요,"안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."
1,2025-10-27,뱅크셀러드,3.0,요쯤 카드나 은행 연동이 잘 안됩니다 카드를 사용 하면 내역이 자동으로 등록되서 올...,"안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."
2,2025-10-26,뱅크셀러드,5.0,5년 넘게 제 가계부로 잘 쓰고 있습니다. 항상 잘 쓰고 있습니다. 추가 되었으면 ...,안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 만족스럽게 이용해 주시는 것 ...
3,2025-10-25,뱅크셀러드,1.0,"업데이트 후 메모•태그 입력하는데 오래걸리고, 자주 오류나서 불편합니다. 메모 기능...","안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."
4,2025-10-25,뱅크셀러드,1.0,업데이트후 계속 꺼짐 실행시키면 계속 꺼집니다 인터넷이 원활하지 않다는 메세지도 가...,"안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."
5,2025-10-23,뱅크셀러드,1.0,업데이트 후 계속 버벅이고 오류나요.,"안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."
6,2025-10-23,뱅크셀러드,4.0,국민연금 납부액과 예상수령액도 연동되게해주세요,안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드에 국민연금을 연결해 납부액과 예...
7,2025-10-22,뱅크셀러드,4.0,"너무너무 잘쓰고 있었는데 코나아이, 투자 부분에 대한 관리가 굉장히 미흡합니다 코나...","안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 애정하시고, 소중한 의견 전달..."
8,2025-10-20,뱅크셀러드,4.0,취소 내역이 섞여 있으면 가계부가 뭔가 미묘하게 맞지 않는 것 같습니다 음...,"안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."
9,2025-10-18,뱅크셀러드,4.0,최근 로그인 한 지 얼마 지나지 않았는데도 장시간 이용이 없어서 로그아웃 되었다고 ...,"안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025-10-26,핀다,5.0,주말에 자금이 필요했는데 1금융에 빠른 대출로 감사드립니다.,"안녕하세요, 최재경님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼..."
1,2025-10-23,핀다,5.0,생각지도 못한자금이 생겼네요 다른곳에선 이한도 이이자율 안나왔는데 핀다 짱이네요,"안녕하세요, 안은경님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼..."
2,2025-10-23,핀다,5.0,최고예요.은행.보험도 안되는걸 판다는 해내요!!,"안녕하세요, 더키님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼저..."
3,2025-10-21,핀다,1.0,저는대출이낮다고거절돼요,"안녕하세요, 장윤정님. 절박한 상황에서 대출이 거절되어 많이 실망하셨을 것 같아 죄..."
4,2025-10-19,핀다,1.0,신분증 찰영 프로그램 정말 쓰레기네요 몇백번을 찍어도 인식을 못하는,"안녕하세요 이승원 고객님, 핀다 고객경험팀입니다. 대출 신청 중 신분증 촬영이 되지..."
5,2025-10-16,핀다,5.0,핀다에서 대출500백받았습니다 감사합니다,"안녕하세요, 전현숙님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼..."
6,2025-10-15,핀다,5.0,이자율도 싸고. 빠르고 편리하게. 받았어요 핀다에. 감사해요,"안녕하세요, 최종도님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼..."
7,2025-10-14,핀다,1.0,신용정보 검색을 자꾸 시키는 어이없네 대출 막으려고 하는짓인,"안녕하세요 sunny k 고객님, 신용조회 오해에 대해 안내드립니다. 핀다는 신용조..."
8,2025-10-14,핀다,4.0,힘들때ㅡ감사힙니다,"안녕하세요, 김선오님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼..."
9,2025-10-14,핀다,3.0,앱을 다운 받고 실행하는데 앱을 종료하였습니다 라고 뜨는데 오류인건가여 대출 알아 ...,"심지혜 고객님 안녕하세요, 핀다 고객경험팀입니다. 핀다 앱을 실행하는 과정에서 앱이..."


In [ ]:
review_list = driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe")
for review in review_list:
    print(driver.find_elementｓ(By.CSS_SELECTOR,"div.RHo1pe")[0].get_attribute('innerHTML'))

In [15]:
app[0]

'핀다'